In [1]:
#read in the full dataset
# Load the metadata TSV file into a DataFrame
file_path = '\\Users\\bscag\\novelty\\becca\\LitMetadataWithS2 (3).tsv'

import pandas as pd
meta = pd.read_csv(file_path, sep='\t')
import requests

In [2]:
import pickle

#load the model
# import joblib
# model = joblib.load(".ipynb_checkpoints/models")

#this reads in the model we had created

#later to run the model we will need this:
# probabilities = model.predict_proba(all_books_df)


with open('viaf_classifier_sept23.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [3]:
# Ensure the authors column is of type list
meta['author'] = meta['authors'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Use the explode method to expand the authors column
meta_exploded = meta.explode('authors')

meta = meta_exploded

# Display the exploded DataFrame
# print(meta_exploded)


In [4]:
df = meta

In [5]:
df['author'] = df['authors']

In [6]:

#normalize the text
import re

def normalize_text(text):
    """
    Normalize the text by converting to lowercase, preserving commas and spaces,
    and removing unnecessary special characters.
    """
    try:
        text = str(text)
        text = text.lower()  # Convert to lowercase
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        text = re.sub(r'[^\w\s,]', '', text)  # Remove all characters except alphanumeric, spaces, and commas
        return text.strip()  # Trim leading and trailing spaces
    except:
        return ''
# df['normalized_author'] = df['author'].apply(normalize_text)
df['normalized_author'] = ""

for idx, row in df.iterrows():
    text = row['author']
    normalized_text = normalize_text(text)
    df.at[idx,'normalized_author'] = normalized_text

In [7]:
df

,doi,journal,year,authors,title,language,wordcount,doctype,citation_counts,S2titles,S2years,condition,matchstrength,S2journal,S2authors,new_cite_count,make_embeddings,paperId,author,normalized_author
0,10.2307/512209,The Review of English Studies,1969,['C. F. Williamson'],Review Article,['eng'],572,article | book-review,0,NaN,no date,not citable,0.0,NaN,NaN,0,0,NaN,['C. F. Williamson'],c f williamson
1,10.2307/437950,Modern Philology,1979,['David Halliburton'],Review Article,['eng'],1863,article | book-review,0,NaN,no date,not citable,0.0,NaN,NaN,0,0,NaN,['David Halliburton'],david halliburton
2,10.2307/461288,PMLA,1967,['Jean-Jacques Demorest'],Pascal et le déséquilibre,['fre'],4999,article | research-article,0,Pascal et le déséquilibre,1967-05-01,no strong match,0.0,NaN,NaN,0,1,3cfbc1e6e654451d97abe60a132d0047841fb0fe,['Jean-Jacques Demorest'],jeanjacques demorest
3,10.2307/3661498,The Review of English Studies,2003,['Robert Douglas-Fairhurst'],Review Article,['eng'],1140,article | book-review,0,NaN,no date,not citable,0.0,NaN,NaN,0,0,NaN,['Robert Douglas-Fairhurst'],robert douglasfairhurst
4,10.2307/3718589,The Modern Language Review,1951,['R. Weiss'],Review Article,['eng'],669,article | book-review,0,NaN,no date,not citable,0.0,NaN,NaN,0,0,NaN,['R. Weiss'],r weiss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71897,10.2307/459445,PMLA,1948,['Elizabeth Nitchie'],"Shelley's ""Hymn to Intellectual Beauty""",['eng'],1059,article | research-article,6,Shelley's “Hymn to Intellectual Beauty”,1948-06-01,already found,0.0,NaN,NaN,6,1,e3a7a04cc735f64d30f500900de93691986ac8e6,['Elizabeth Nitchie'],elizabeth nitchie
71898,10.2307/3715352,The Modern Language Review,1931,['K. G. T. Webster'],"The Water-Bridge in Chrétien's ""Charrette""",['eng'],2833,article | research-article,1,"The Water-Bridge in Chretien's ""Charrette""",NaN,already found,0.0,NaN,NaN,1,1,a4cfc0af1df2fc4744da5ef5170e89bd3d325952,['K. G. T. Webster'],k g t webster
71899,10.2307/468933,New Literary History,1980,['anonymous'],Front Matter,['eng'],2515,article | misc,0,NaN,no date,not citable,0.0,NaN,NaN,0,0,NaN,['anonymous'],anonymous
71900,10.2307/2871890,ELH,1955,['J. Hillis Miller'],The Creation of the Self in Gerard Manley Hopkins,['eng'],10566,article | research-article,7,The Creation of the Self in Gerard Manley Hopkins,1955-12-01,already found,0.0,NaN,NaN,7,1,0f3ac90ec912c115f4c174ee7e7f500b521db3aa,['J. Hillis Miller'],j hillis miller


In [8]:
from ast import literal_eval 

rows = []
for index, row in df.iterrows():
    author = row['author']
    author = normalize_text(author)
    # author = literal_eval(author)
    if ',' in author:
        author_list = author.split(',')
        # author_list = literal_eval(author_list)
        for author in author_list:
            new_row = {
             'author': author,
            'journal': row['journal'],
            'year': row['year'],
            'title': row['title'],
            'S2titles': row['S2titles'],
            'S2Years': row['S2years'],
            
        }
        rows.append(new_row)
    else:
        row_orig = {     
            'author': row['author'],
            'journal': row['journal'],
            'year': row['year'],
            'title': row['title'],
            'S2titles': row['S2titles'],
            'S2Years': row['S2years'],
            
        }
        rows.append(row_orig)

In [9]:
print(author_list)

['yvonne noble', ' john b humma']


In [10]:
rows

[{'author': "['C. F. Williamson']",
  'journal': 'The Review of English Studies',
  'year': 1969,
  'title': 'Review Article',
  'S2titles': nan,
  'S2Years': 'no date'},
 {'author': "['David Halliburton']",
  'journal': 'Modern Philology',
  'year': 1979,
  'title': 'Review Article',
  'S2titles': nan,
  'S2Years': 'no date'},
 {'author': "['Jean-Jacques Demorest']",
  'journal': 'PMLA',
  'year': 1967,
  'title': 'Pascal et le déséquilibre',
  'S2titles': 'Pascal et le déséquilibre',
  'S2Years': '1967-05-01'},
 {'author': "['Robert Douglas-Fairhurst']",
  'journal': 'The Review of English Studies',
  'year': 2003,
  'title': 'Review Article',
  'S2titles': nan,
  'S2Years': 'no date'},
 {'author': "['R. Weiss']",
  'journal': 'The Modern Language Review',
  'year': 1951,
  'title': 'Review Article',
  'S2titles': nan,
  'S2Years': 'no date'},
 {'author': "['Dennis R. Danielson']",
  'journal': 'ELH',
  'year': 1980,
  'title': 'Imago Dei, "Filial Freedom," and Miltonic Theodicy',
  

In [11]:
expanded_df = pd.DataFrame(rows)

In [12]:
expanded_df

,author,journal,year,title,S2titles,S2Years
0,['C. F. Williamson'],The Review of English Studies,1969,Review Article,NaN,no date
1,['David Halliburton'],Modern Philology,1979,Review Article,NaN,no date
2,['Jean-Jacques Demorest'],PMLA,1967,Pascal et le déséquilibre,Pascal et le déséquilibre,1967-05-01
3,['Robert Douglas-Fairhurst'],The Review of English Studies,2003,Review Article,NaN,no date
4,['R. Weiss'],The Modern Language Review,1951,Review Article,NaN,no date
...,...,...,...,...,...,...
71897,['Elizabeth Nitchie'],PMLA,1948,"Shelley's ""Hymn to Intellectual Beauty""",Shelley's “Hymn to Intellectual Beauty”,1948-06-01
71898,['K. G. T. Webster'],The Modern Language Review,1931,"The Water-Bridge in Chrétien's ""Charrette""","The Water-Bridge in Chretien's ""Charrette""",NaN
71899,['anonymous'],New Literary History,1980,Front Matter,NaN,no date
71900,['J. Hillis Miller'],ELH,1955,The Creation of the Self in Gerard Manley Hopkins,The Creation of the Self in Gerard Manley Hopkins,1955-12-01


In [13]:
# df_combined = pd.concat([df, expanded_df])

In [14]:
# df_combined

In [15]:
# columns_to_drop = ['doi','authors','title','language','wordcount','doctype','citation_counts','matchstrength','S2journal','S2authors','new_cite_count','make_embeddings','S2Years']
# df_combined.drop(columns_to_drop, axis=1, inplace=True)

In [16]:
df = expanded_df

In [17]:
#get the VIAF entries for all data
def get_name(name):
    params = {
        'query' : f'local.personalNames = "{name}"',
        'maximumRecords': 10,
        'startRecord' : 1,
        'sortKeys': 'holdingscount',
        'httpAccept': 'application/json'
    }

    headers={'User-Agent': user_agent}
    url = "https://viaf.org/viaf/search"

    r = requests.get(url,params=params,headers=headers, timeout=10)
    data = r.json()

    return data

In [18]:
user_agent = 'YOUR PROJECT NAME HERE'
id_column_name = "Name"

pause_between_req = 1

use_title_reconcilation = True

cache = {}

In [19]:
import pandas as pd
processing_errors = []
from fuzzywuzzy import fuzz

# Function to perform search and return DataFrame with search results
def search_author(author_name):
    # Initialize list to store search results
    search_results = []

    try:
        data = get_name(author_name)
        records = data['searchRetrieveResponse']['records']
        total_records = len(records)

        for idx, record in enumerate(records):
            record_data = record['record']['recordData']
            birthdate = record['record']['recordData']['birthDate']
            viaf_title_list = []
            # Extract titles if available
            if 'titles' in record_data:
                titles_data = record_data['titles']
                if titles_data is not None:
                    if isinstance(titles_data['work'], list):
                        # Extract titles from the list of works
                        viaf_title_list.extend([work['title'] for work in titles_data['work'] if 'title' in work])
                    else:
                        # Extract title from a single work
                        title = titles_data['work'].get('title')
                        if title:
                            viaf_title_list.append(title)

#             # Get S2 titles for the author
#             s2_titles = birthdates_df.loc[birthdates_df['author'] == author_name, 'S2 Titles'].iloc[0]
#             if isinstance(s2_titles, float):
#                 s2_titles = []

#             # Perform fuzzy title matching
#             title_matched = False
#             matched_title = None
#             for title_from_viaf in title_list:
#                 for s2_title in s2_titles:
#                     if fuzz.partial_ratio(s2_title, title_from_viaf) >= 70:
#                         title_matched = True
#                         matched_title = s2_title
#                         break
#                 if title_matched:
#                     break

#             # Append author name, title list, S2 titles, title matched, matched title, and birthdate to search results
#             # birthdate = birthdates_df.loc[birthdates_df['author'] == author_name, 'birthdate'].iloc[0]
#             search_results.append({'author': author_name, 'record_count': len(records),
#                                    'record_enumerated': idx, 'title_list': title_list,
#                                    'S2 Titles': s2_titles, 'title_matched': title_matched,
#                                    'matched_title': matched_title, 'birthdate': birthdate})
            
            # Append author name, title list, S2 titles, title matched, matched title, and birthdate to search results
            # birthdate = birthdates_df.loc[birthdates_df['author'] == author_name, 'birthdate'].iloc[0]
            title_list = df.loc[df['author'] == author_name, 'title_list'].iloc[0]

            search_results.append({'author': author_name, 'record_count': len(records),
                                   'record_enumerated': idx, 'viaf_title_list': viaf_title_list,
                                   'birthdate': birthdate,'title_list':title_list})
    except Exception as e:
         processing_errors.append(f'Processing Error for {author_name}: {e}')

    return search_results



In [20]:
# result_df = df
# # result_df['VIAF_titlelist'] = result_df['record_enumerated_titles'] 
# result_df['S2_titlelist'] = result_df['S2titles']
# result_df['average_S2_pubdate'] = result_df['avg_pubdates']
# result_df['VIAF_birthdate'] = result_df['standard_birthdate']
# result_df['S2_Author'] = result_df['author']

df['title_list'] = df['S2titles']
# df = result_df

In [21]:
len(df)

71902

In [22]:
#fix lists of multiple author names...


In [23]:
# df = df_combined

In [24]:
#we don't want the VIAF search to take years... so let's get a unique list of author names

unique_author_names = []

for author_name in df['author']:
    if author_name not in unique_author_names:
        unique_author_names.append(author_name)

In [25]:
len(unique_author_names)

22377

In [26]:
# # Iterate through each author name in the original DataFrame and perform the search
# all_search_results = []
# row_counter = 0  # Counter to limit the number of rows processed
# 
# # for author_name in df['author']:
# for author_name in unique_author_names:
#     search_results = search_author(author_name)
#     all_search_results.extend(search_results)
#     # row_counter += 1
#     # if row_counter >= 5:  # Adjust the number as needed
#     #     break
# 
# # Convert list of dictionaries to DataFrame
# all_search_results_df\
#     = pd.DataFrame(all_search_results)
# # print(all_search_results_df)


In [27]:
import pandas as pd
import os

# Path to save the results
results_file = 'search_results_2.csv'

# Load existing results if the file exists
if os.path.exists(results_file):
    all_search_results_df = pd.read_csv(results_file)
else:
    all_search_results_df = pd.DataFrame()

# Get the unique author names, skipping those already processed
if not all_search_results_df.empty:
    processed_authors = all_search_results_df['author'].unique()
    unique_author_names = [name for name in unique_author_names if name not in processed_authors]

# Chunk processing parameters
chunk_size = 2000  # Adjust the size as needed
total_authors = len(unique_author_names)

# Process in chunks
for i in range(0, total_authors, chunk_size):
    chunk = unique_author_names[i:i + chunk_size]
    all_search_results = []

    for author_name in chunk:
        search_results = search_author(author_name)
        all_search_results.extend(search_results)

    # Convert list of dictionaries to DataFrame
    chunk_df = pd.DataFrame(all_search_results)

    # Append to existing results
    all_search_results_df = pd.concat([all_search_results_df, chunk_df], ignore_index=True)

    # Save the updated results to the file
    all_search_results_df.to_csv(results_file, index=False)

    print(f"Processed authors {i + 1} to {min(i + chunk_size, total_authors)}")


Processed authors 1 to 2000
Processed authors 2001 to 4000
Processed authors 4001 to 6000
Processed authors 6001 to 8000
Processed authors 8001 to 10000
Processed authors 10001 to 12000
Processed authors 12001 to 14000
Processed authors 14001 to 15045


In [28]:
all_search_results

[{'author': "['Anne Barbeau Gardiner']",
  'record_count': 2,
  'record_enumerated': 0,
  'viaf_title_list': ["Ancient faith and modern freedom in John Dryden's The hind and the panther",
   "The intellectual design of John Dryden's heroic plays, c1970:",
   'Malice defeated, 1988:',
   'matchless rogue'],
  'birthdate': '1938',
  'title_list': 'A Witty French Preacher in the English Court, Dryden, and the Great Debate on the Real Presence, 1661-1688'},
 {'author': "['Anne Barbeau Gardiner']",
  'record_count': 2,
  'record_enumerated': 1,
  'viaf_title_list': [],
  'birthdate': '0',
  'title_list': 'A Witty French Preacher in the English Court, Dryden, and the Great Debate on the Real Presence, 1661-1688'},
 {'author': "['Kate E. Tunstall']",
  'record_count': 1,
  'record_enumerated': 0,
  'viaf_title_list': ['Blindness and enlightenment : an essay',
   'Diderot and Rousseau : networks of Enlightenment',
   'Diderot\'s "promenade Vernet", or the Salon as landscape garden',
   "Les di

In [29]:

len(unique_author_names
    )

15045

In [30]:
len(all_search_results)

3136

In [31]:
all_search_results_df\
    = pd.DataFrame(all_search_results)

In [32]:
#71902... did it keep going??

In [33]:
all_search_results_df

,author,record_count,record_enumerated,viaf_title_list,birthdate,title_list
0,['Anne Barbeau Gardiner'],2,0,[Ancient faith and modern freedom in John Dryd...,1938,"A Witty French Preacher in the English Court, ..."
1,['Anne Barbeau Gardiner'],2,1,[],0,"A Witty French Preacher in the English Court, ..."
2,['Kate E. Tunstall'],1,0,"[Blindness and enlightenment : an essay, Dider...",2000,NaN
3,['Kevin McLaughlin'],7,0,"[The arcades project, Paperwork : fiction and ...",1959,NaN
4,['Kevin McLaughlin'],7,1,[Online ad auctions: an experiment],0,NaN
...,...,...,...,...,...,...
3131,['John T. Matthews'],3,0,[Anxious souls will ask... : the Christ-center...,1950-03-21,The Marriage of Speaking and Hearing in Absalo...
3132,['John T. Matthews'],3,1,"[Anxious souls will ask--, 2005:, Bonhoeffer :...",1949-11-19,The Marriage of Speaking and Hearing in Absalo...
3133,['John T. Matthews'],3,2,[],0,The Marriage of Speaking and Hearing in Absalo...
3134,['Melissa J. Homestead'],1,0,"[American women authors and literary property,...",1963-04-13,NaN


In [34]:
all_search_results_df.tail()

,author,record_count,record_enumerated,viaf_title_list,birthdate,title_list
3131,['John T. Matthews'],3,0,[Anxious souls will ask... : the Christ-center...,1950-03-21,The Marriage of Speaking and Hearing in Absalo...
3132,['John T. Matthews'],3,1,"[Anxious souls will ask--, 2005:, Bonhoeffer :...",1949-11-19,The Marriage of Speaking and Hearing in Absalo...
3133,['John T. Matthews'],3,2,[],0,The Marriage of Speaking and Hearing in Absalo...
3134,['Melissa J. Homestead'],1,0,"[American women authors and literary property,...",1963-04-13,NaN
3135,['Barrie Ruth Straus'],1,0,"[The Catholic Church, c1986:]",0,The Subversive Discourse of the Wife of Bath: ...


In [35]:
all_search_results_df.to_csv('all_search_results_df_18hr_sept17.csv')

In [36]:
all_search_results

[{'author': "['Anne Barbeau Gardiner']",
  'record_count': 2,
  'record_enumerated': 0,
  'viaf_title_list': ["Ancient faith and modern freedom in John Dryden's The hind and the panther",
   "The intellectual design of John Dryden's heroic plays, c1970:",
   'Malice defeated, 1988:',
   'matchless rogue'],
  'birthdate': '1938',
  'title_list': 'A Witty French Preacher in the English Court, Dryden, and the Great Debate on the Real Presence, 1661-1688'},
 {'author': "['Anne Barbeau Gardiner']",
  'record_count': 2,
  'record_enumerated': 1,
  'viaf_title_list': [],
  'birthdate': '0',
  'title_list': 'A Witty French Preacher in the English Court, Dryden, and the Great Debate on the Real Presence, 1661-1688'},
 {'author': "['Kate E. Tunstall']",
  'record_count': 1,
  'record_enumerated': 0,
  'viaf_title_list': ['Blindness and enlightenment : an essay',
   'Diderot and Rousseau : networks of Enlightenment',
   'Diderot\'s "promenade Vernet", or the Salon as landscape garden',
   "Les di

In [37]:
def any_neg(birth2maxdate, birth2mindate):
    if birth2maxdate is not None and birth2mindate is not None:
        if birth2maxdate < 0 or birth2mindate < 0:
            return 1
        
    return 0

def title_list_len(title_list):
    return len(title_list) if isinstance(title_list, list) else 0

def author_length(author):
    return len(author)

In [38]:
# df_test = df.head(10)
from ast import literal_eval

for idx, row in df.iterrows():
    birth = row['VIAF_birthdate']
    pubdates = row['S2_pubdates']
    titlelist = row['VIAF_titlelist']
    author = row['author']
    if pubdates is not None:
        print(pubdates)
        # print(birth)
        pubdates = literal_eval(pubdates)
        author = str(row['author'])
        birth2maxdate_value = max(pubdates) - int(birth)
        birth2mindate_value = min(pubdates)- int(birth)
        # any_neg = any_neg(birth2maxdate_value,birth2mindate_value)
        title_list_len = len(titlelist)
        author_len = len(author)
        
        # print(birth2maxdate_value)
        df.at[idx,'birth2max_value'] = birth2maxdate_value
        df.at[idx, 'birth2min_value'] = birth2mindate_value
        # df.at[idx, 'any_neg'] = any_neg
        df.at[idx, 'title_list_len'] = title_list_len
        df.at[idx, 'author_len'] = author_len
        # print(birth2maxdate_value)
    # else:
    #     print(pubdates, birth)
    


KeyError: 'VIAF_birthdate'

In [ ]:
import pandas as pd
import ast


# Function to compute average publication date
def compute_avg_pubdate(pubdates):
    try:
        # Convert the string representation to an actual tuple
        pubdates_tuple = ast.literal_eval(pubdates)
        
        # Check if it's a tuple of integers
        if isinstance(pubdates_tuple, tuple) and all(isinstance(date, int) for date in pubdates_tuple):
            return sum(pubdates_tuple) / len(pubdates_tuple)
        else:
            return None
    except (ValueError, SyntaxError):
        return None

# Apply the function to each row
df['avg_pubdate'] = df['S2_pubdates'].apply(lambda x: compute_avg_pubdate(str(x)))



In [ ]:
#publication_age
df['publication_age'] = df['avg_pubdate'] - df['VIAF_birthdate']

In [ ]:
df['publication_age'] = pd.to_numeric(df['publication_age'], errors='coerce')


In [ ]:
df = df.fillna(0)


In [ ]:
df['status'] = ""

for idx, row in df.iterrows():
    if row['publication_age'] < 0:
        df.at[idx,'status'] = 'not_born'
    elif row['publication_age'] > 100:
        df.at[idx,'status'] = 'zombie'
    elif row['publication_age'] < 9:
        df.at[idx,'status'] = 'toddler'
    else:
        df.at[idx,'status'] = '0'

In [ ]:
df['status'] = df['status'].str.replace('zombie', '1')
df['status'] = df['status'].fillna('0')
df['status'] = df['status'].str.replace('not_born', '2')
df['status'] = df['status'].str.replace('toddler', '3')


Now add VIAF and S2 titlelist embeddings

In [ ]:
#embeddings

from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings for a list of titles
def get_embeddings(titles_list):
    if isinstance(titles_list, str):
        return model.encode(titles_list)
    else:
        return model.encode('')

# Generate embeddings for each row in the DataFrame
df['S2_embeddings'] = df['S2titles'].apply(get_embeddings)
df['VIAF_embeddings'] = df['VIAF_titlelist'].apply(get_embeddings)


#was taking forever to run, so I'm leaving this piece out for now to make sure everything else works!

In [ ]:
import numpy as np
df['S2_embeddings'] = df['S2_embeddings'].apply(np.mean)
df['VIAF_embeddings'] = df['VIAF_embeddings'].apply(np.mean)

In [ ]:
#get overlapping words and overlapping lemmas:
# Function to lemmatize text and return a set of lemmatized words
import spacy
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    doc = nlp(text)
    return set(token.lemma_ for token in doc if not token.is_stop and not token.is_punct)

# Function to calculate lemma overlap and return the overlapping lemmas
def calculate_lemma_overlap(text1, text2):
    text1 = str(text1)
    text2 = str(text2)
    lemmas1 = lemmatize_text(text1)
    lemmas2 = lemmatize_text(text2)
    overlap = lemmas1.intersection(lemmas2)
    return len(overlap), overlap

# # Apply the function to compute lemma overlap for each row in the DataFrame
# df[['lemma_overlap', 'overlapping_lemmas']] = df.apply(
#     lambda row: pd.Series(calculate_lemma_overlap(row['VIAF_titlelist'], row['S2_titlelist'])), 
#     axis=1)
df[['lemma_overlap', 'overlapping_lemmas']] = df.apply(
    lambda row: pd.Series(calculate_lemma_overlap(row['VIAF_titlelist'], row['S2_titlelist'])), 
    axis=1)


In [ ]:
#add overlaps

#add word overlap as a new feature
import pandas as pd
from nltk.corpus import stopwords
import nltk

# Download stopwords from NLTK if you haven't already
nltk.download('stopwords')
# Define stop words
stop_words = set(stopwords.words('english'))

# Function to create bag of words after removing stop words
def bag_of_words(title):
    words = title.lower().split()
    return set([word for word in words if word not in stop_words])

# Function to find and count word overlap
def find_word_overlap(row):
    try:
        v_list = ast.literal_eval(row['VIAF_titlelist'])
        s_list = ast.literal_eval(row['S2_titlelist'])
        if isinstance(v_list, list):

            for title in v_list:
                title = str(title)
        if isinstance(s_list, list):

            for title in s_list:
                title = str(title)
        if isinstance(v_list, list):

            v_bag = bag_of_words(v_list) 
        if isinstance(s_list, list):

            s_bag = bag_of_words(s_list) 

        overlap = v_bag & s_bag
    except:
        overlap = ''
    return pd.Series([len(overlap), list(overlap)])
  
# Apply the function to each row and create two new columns
df[['word_overlap_count', 'overlapping_words']] = df.apply(find_word_overlap, axis=1)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (precision_recall_curve, roc_auc_score,
                             confusion_matrix, ConfusionMatrixDisplay,
                             classification_report, accuracy_score)
import matplotlib.pyplot as plt


X = df.drop(columns=['author','VIAF_titlelist','S2_titlelist','overlapping_words','selected_birthyear','overlapping_lemmas','title_list','record_enumerated_titles','S2Titles','S2titles','matched_title_list','common_words','notes','normalized_author','S2_Titlelist','S2_Author','S2_pubdates','mean_embedding','matched_title?','match?','match'])  # Drop the label and metadata columns
y = df['match?']

# Optional: If you want to split into training and test sets first
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Initialize the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform cross-validation
model = RandomForestClassifier()
y_pred_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba')

# Store predictions with original indices
predictions_df = pd.DataFrame(y_pred_proba, index=original_indices, columns=['Class_0_Proba', 'Class_1_Proba'])

# Merge the metadata back
results_df = pd.concat([original_metadata, predictions_df], axis=1)
results_df['True_Label'] = y.loc[results_df.index]

# Now you have a DataFrame with metadata and predictions
print(results_df.head())

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import Binarizer

# Initialize the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform cross-validation to get predicted probabilities
y_pred_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba')

# Store predictions with original indices
predictions_df = pd.DataFrame(y_pred_proba, index=original_indices, columns=['Class_0_Proba', 'Class_1_Proba'])

# Merge the metadata back
results_df = pd.concat([original_metadata, predictions_df], axis=1)
results_df['True_Label'] = y.loc[results_df.index]

# Define the threshold for decision
threshold = 0.5  # You can change this value based on your needs

# Compute class decisions based on the threshold
results_df['Class_Decision'] = (results_df['Class_1_Proba'] > threshold).astype(int)

# Now you have a DataFrame with metadata, predictions, and class decisions
print(results_df.head())


In [ ]:
#find the optimal threshold
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

# Initialize the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform cross-validation to get predicted probabilities
y_pred_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba')

# Store predictions with original indices
predictions_df = pd.DataFrame(y_pred_proba, index=original_indices, columns=['Class_0_Proba', 'Class_1_Proba'])

# Define the threshold for decision
threshold = 0.8  # You can change this value based on your needs

# Compute class decisions based on the threshold
predictions_df['Class_Decision'] = (predictions_df['Class_1_Proba'] > threshold).astype(int)

# Merge the metadata back
results_df = pd.concat([original_metadata, predictions_df], axis=1)
results_df['True_Label'] = y.loc[results_df.index]

# Generate class predictions
y_pred = (y_pred_proba[:, 1] > threshold).astype(int)  # Use the same threshold for predictions

# Print the DataFrame with metadata, predictions, and class decisions
print(results_df.head())

# Generate and print the classification report
report = classification_report(y, y_pred)
print("\nClassification Report:\n", report)
